In [ ]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback


In [ ]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


def subdivide(tname):
    c = "CREATE TABLE IF NOT EXISTS %s_subdivided AS SELECT ST_SubDivide(wkb_geometry) AS geom, objectid as %s_gid FROM %s;" % (tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd
    
def create_index(tname):
    c = "CREATE UNIQUE INDEX IF NOT EXISTS %s_idx ON wdpa_%s_intersection_areas (wdpa_gid, %s_gid);" % (tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

def vacuum_table(tname):
    c = "VACUUM ANALYZE wdpa_%s_intersection_areas;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   
    

def select_into(tname):
    c = """
    SELECT 
      wdpa_gid, 
      %s_gid,  
      SUM(intersection_area) as intersection_area
    INTO TABLE wdpa_%s_intersection_areas
    FROM wdpa_%s_subdivided 
    GROUP BY wdpa_gid, %s_gid
    """
    c = c % (tname, tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd

def get_counts(tname):
    print tname
    c = """
    SELECT 
      COUNT(*)
    FROM wdpa_%s_intersection_areas;
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd



In [ ]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name.lower())
tnames.sort()

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(select_into, tname)
exe.shutdown()
None

In [ ]:
for tname in tnames:
    get_counts(tname)

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(create_index, tname)
exe.shutdown()
None

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(vacuum_table, tname)
exe.shutdown()
None